In [1]:
import pandas as pd
import numpy as np
df=pd.read_excel("/content/sample data.xlsx")
print(df.head())

                        concept(gold)  concept_id
0  history of total lobectomy of lung     4005751
1                  open heart surgery     4094240
2               intestinal absorption     2721192
3                          loperamide      991876
4                              anemia      381839


In [24]:
!pip install transformers

In [2]:
from transformers import BertModel, BertTokenizer

In [3]:
# model_name='clinicalbert-base'
#model_name='emilyalsentzer/Bio_ClinicalBERT'
model_name='NeuML/pubmedbert-base-embeddings'

In [4]:
tokenizer=BertTokenizer.from_pretrained(model_name)
#tokenizer used to splt the given text into tokens
model=BertModel.from_pretrained(model_name)

In [5]:
tokenizer
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [3]:
import pandas as pd

csv_file_path = "CONCEPT.csv"

df = pd.read_csv(csv_file_path, sep='\t', quoting=3, header=None)
df.columns = ["concept_id", "concept_name", "domain_id", "vocabulary_id", "concept_class_id", "standard_concept", "concept_code", "valid_start_date", "valid_end_date", "invalid_reason"]

df_selected = df[['concept_id', 'concept_name']]

print(df_selected.head())


C:\Users\annem\AppData\Local\Temp\ipykernel_22852\3162721891.py:5: DtypeWarning: Columns (0,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path, sep='\t', quoting=3, header=None)


   concept_id                             concept_name
0  concept_id                             concept_name
1    45756805                     Pediatric Cardiology
2    45756804                 Pediatric Anesthesiology
3    45756803  Pathology-Anatomic / Pathology-Clinical
4    45756802                    Pathology - Pediatric


In [14]:
import torch

embeddings = []

for concept_name in df_selected['concept_name'][:50]:
   
    if isinstance(concept_name, str):
        input_ids = tokenizer(concept_name, return_tensors='pt').input_ids

        with torch.no_grad():
            concept_embedding = model(input_ids)['last_hidden_state'].mean(dim=1).squeeze().numpy()

        embeddings.append(concept_embedding)
    else:
        print(f"Skipping concept_name: {concept_name} as it is not a valid string.")


In [15]:
# Create the Faiss index
dimension = len(embedding_array[0])
index = faiss.IndexFlatL2(dimension)
index.add(embedding_array)

# Save the Faiss index
faiss.write_index(index, "vector_index.index")

In [ ]:
import faiss
import numpy as np

embeddings = []
concept_names = []

for concept_name in df_selected['concept_name'][:10]:
    if isinstance(concept_name, str):
        input_ids = tokenizer(concept_name, return_tensors='pt').input_ids

        with torch.no_grad():
            concept_embedding = model(input_ids)['last_hidden_state'].mean(dim=1).squeeze().numpy()

        embeddings.append(concept_embedding)
        concept_names.append(concept_name)
    else:
        print(f"Skipping concept_name: {concept_name} as it is not a valid string.")

# Convert the embeddings list to a numpy array
embedding_array = np.array(embeddings, dtype=np.float32)

# Create the Faiss index
dimension = len(embedding_array[0])
index = faiss.IndexFlatL2(dimension)
index.add(embedding_array)

# Save the Faiss index
faiss.write_index(index, "vector_index.index")

# Save the concept names for each index
np.save("concept_names.npy", np.array(concept_names))

In [ ]:
import faiss
import torch

loaded_index = faiss.read_index("vector_index.index")
concept_names = np.load("concept_names.npy", allow_pickle=True)

new_entity_name = "  Pediatric Cardiolosgy"  # Pass the entity here
new_entity_input_ids = tokenizer(new_entity_name, return_tensors='pt').input_ids  # Tokenize the new entity

with torch.no_grad():
    new_entity_embedding = model(new_entity_input_ids)['last_hidden_state'].mean(dim=1).squeeze().numpy()  # Obtain the embedding

k = 1  # Retrieve top 1
distance, concept_indices = loaded_index.search(new_entity_embedding.reshape(1, -1), k)

# Retrieve and print the actual concept names using the mapped indices
related_concepts = [concept_names[idx] for idx in concept_indices[0]]

print("Related Concepts for New Entity:", new_entity_name)
for concept in related_concepts:
    print(concept)




In [8]:
import torch
embeddings = []
for concept_name in df_selected['concept_name']:
    input_ids = tokenizer(concept_name, return_tensors='pt').input_ids

    with torch.no_grad():
        concept_embedding = model(input_ids)['last_hidden_state'].mean(dim=1).squeeze().numpy()

    embeddings.append(concept_embedding)

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
import torch

embeddings = []

for concept_name in df_selected['concept_name']:
   
    if isinstance(concept_name, str):
        input_ids = tokenizer(concept_name, return_tensors='pt').input_ids

        with torch.no_grad():
            concept_embedding = model(input_ids)['last_hidden_state'].mean(dim=1).squeeze().numpy()

        embeddings.append(concept_embedding)
    else:
        print(f"Skipping concept_name: {concept_name} as it is not a valid string.")



Skipping concept_name: nan as it is not a valid string.
Skipping concept_name: nan as it is not a valid string.


In [ ]:
pip install faiss-cpu


In [ ]:
# import faiss
# import numpy as np

# # Assuming 'embeddings' contains the embeddings and 'concept_names' contains corresponding concept names
# # If the concept names are available separately
# concept_names = df_selected['concept_name'].tolist()  # Retrieve concept names from the DataFrame

# dimension = len(embeddings[0])
# index = faiss.IndexFlatL2(dimension)  # Use Euclidean distance for the index
# embedding_array = np.array(embeddings, dtype=np.float32)
# index.add(embedding_array)

# # Create a mapping between indices and concept names
# index_to_concept_map = {i: name for i, name in enumerate(concept_names)}

# # Save the index and the mapping for retrieval phase
# faiss.write_index(index, "vector_index.index")
# np.save("index_to_concept_map.npy", index_to_concept_map)


In [10]:
import faiss
import numpy as np
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)#used Euclidian distance
embedding_array = np.array(embeddings, dtype=np.float32)
index.add(embedding_array)
faiss.write_index(index, "vector_index.index")

In [14]:
loaded_index = faiss.read_index("vector_index.index")
new_entity_name = "lungs of lobes"#pass entity here
new_entity_input_ids = tokenizer(new_entity_name, return_tensors='pt').input_ids #tokeni
with torch.no_grad():
    new_entity_embedding = model(new_entity_input_ids)['last_hidden_state'].mean(dim=1).squeeze().numpy()#embedding

k = 1 #top1retreiving
distance, concept_indices = loaded_index.search(new_entity_embedding.reshape(1, -1), k)
related_concepts = [df_selected.iloc[idx]['concept_name'] for idx in concept_indices[0]]

print("Related Concepts for New Entity:", new_entity_name)
for concept in related_concepts:
    print(concept)


Related Concepts for New Entity: lungs of lobes
Pathology-Anatomic / Pathology-Clinical


In [ ]:
import pandas as pd
import numpy as np
import torch
import faiss

loaded_index = faiss.read_index("vector_index.index")

input_file = "Usagi_eval.xlsx"
output_file = "output_results.xlsx"

input_df = pd.read_excel(input_file)

results = []

for index, row in input_df.iterrows():
    entity = row['Entity']
    ground_truth = row['Concept(gold)']
    
    #  'entity' is a valid string
    if isinstance(entity, str):
        input_ids = tokenizer(entity, return_tensors='pt').input_ids
        with torch.no_grad():
            entity_embedding = model(input_ids)['last_hidden_state'].mean(dim=1).squeeze().numpy()

        k = 1  # Retrieve the top 1 concept
        distance, concept_indices = loaded_index.search(entity_embedding.reshape(1, -1), k)

        related_concept = df_selected.iloc[concept_indices[0][0]]['concept_name']

        results.append([entity, ground_truth, related_concept])
    else:
        print(f"Skipping entity: {entity} as it is not a valid string.")

results_df = pd.DataFrame(results, columns=['Entity', 'GroundTruth', 'RelatedConcept'])

results_df.to_excel(output_file, index=False)


In [12]:
import pandas as pd
import numpy as np
import torch
import faiss

loaded_index = faiss.read_index("vector_index.index")

input_file = "Usagi_eval.xlsx"
output_file = "output_results.xlsx"

input_df = pd.read_excel(input_file)

results = []


for index, row in input_df.head(20).iterrows():
    entity = row['Entity']
    ground_truth = row['Concept(gold)']
    
   
    if isinstance(entity, str):
        input_ids = tokenizer(entity, return_tensors='pt').input_ids
        with torch.no_grad():
            entity_embedding = model(input_ids)['last_hidden_state'].mean(dim=1).squeeze().numpy()

        k = 1  
        distance, concept_indices = loaded_index.search(entity_embedding.reshape(1, -1), k)

        related_concept = df_selected.iloc[concept_indices[0][0]]['concept_name']

        results.append([entity, ground_truth, related_concept])
    else:
        print(f"Skipping entity: {entity} as it is not a valid string.")

results_df = pd.DataFrame(results, columns=['Entity', 'GroundTruth', 'RelatedConcept'])

results_df.to_excel(output_file, index=False)


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


output_file = "output_results.xlsx"
results_df = pd.read_excel(output_file)

ground_truth = results_df['GroundTruth']
predicted_concepts = results_df['RelatedConcept']


accuracy = accuracy_score(ground_truth, predicted_concepts)

precision = precision_score(ground_truth, predicted_concepts, average='micro')
recall = recall_score(ground_truth, predicted_concepts, average='micro')
f1 = f1_score(ground_truth, predicted_concepts, average='micro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


In [ ]:
# mean averages
#.squeeze() removes any redundant dimensions,
#converting the tensor into a one-dimensional tensor.

In [22]:
# embeddings[0]

In [27]:
# import faiss
# index = faiss.IndexFlatL2(embeddings[0].shape[0])


In [29]:
# embedding_array = np.array(embeddings, dtype=np.float32)
# index.add(embedding_array)

In [32]:
# embedding_array

array([[ 0.11773689,  0.16933465,  0.28359666, ...,  0.6540919 ,
        -0.29623917,  0.37046832],
       [-0.61652166,  0.27194324, -0.85910165, ..., -0.21059635,
         0.66623867, -0.02411818],
       [-1.2852491 , -0.2501228 , -0.4878367 , ..., -0.0978955 ,
        -0.52875453, -0.18915473],
       ...,
       [-0.63698983, -0.04616853, -0.5410171 , ...,  0.13265307,
        -0.23601161, -0.8517514 ],
       [-1.2534174 ,  1.1010355 ,  0.32653096, ..., -0.40512815,
         0.17378646,  0.30994403],
       [-0.35723314, -0.4312334 , -0.6614497 , ..., -0.04284064,
        -0.33909023, -0.07046733]], dtype=float32)

In [37]:
# new_entity_name = "lobes of the lung"
# new_entity_input_ids = tokenizer(new_entity_name, return_tensors='pt').input_ids

# with torch.no_grad():
#     new_entity_embedding = model(new_entity_input_ids)['last_hidden_state'].mean(dim=1).squeeze().numpy()

# k = 1
# distance, concept_indices = index.search(new_entity_embedding.reshape(1, -1), k)

# related_concepts = [df.iloc[idx]['concept(gold)'] for idx in concept_indices[0]]

# print("Related Concepts for New Entity:", new_entity_name)
# for concept in related_concepts:
#     print(concept)


Related Concepts for New Entity: lobes of the lung
history of total lobectomy of lung


In [ ]:
# stores in local database
#faiss.write_index(index, "vector_index.index")

In [49]:
print(concept_indices)

[[ 0 16 14 12  1 15 13  5  3  4]]


In [48]:
import faiss

loaded_index = faiss.read_index("vector_index.index")
num_vectors = loaded_index.ntotal
print(num_vectors)
num_to_print = min(num_vectors, 10)

for i in range(num_to_print):
    vector = loaded_index.reconstruct(i)
    #print(f"Vector {i}:", vector)


17


In [11]:
import pandas as pd
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch

# Load CSV file into a pandas DataFrame
csv_file_path = "CONCEPT.csv"
df = pd.read_csv(csv_file_path, sep='\t', quoting=3, header=None)
df.columns = ["concept_id", "concept_name", "domain_id", "vocabulary_id", "concept_class_id", "standard_concept", "concept_code", "valid_start_date", "valid_end_date", "invalid_reason"]

df_selected = df[['concept_id', 'concept_name']]

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")


embeddings = []
concept_names = []
for concept_name in df_selected['concept_name'][:10]:
    if isinstance(concept_name, str):
       
        input_ids = tokenizer(concept_name, return_tensors='pt').input_ids

        with torch.no_grad():
            concept_embedding = model(input_ids)['last_hidden_state'].mean(dim=1).squeeze().numpy()

        embeddings.append(concept_embedding)
        concept_names.append(concept_name)
    else:
        print(f"Skipping concept_name: {concept_name} as it is not a valid string.")
embedding_array = np.array(embeddings, dtype=np.float32)

dimension = len(embedding_array[0])
index = faiss.IndexFlatL2(dimension)
index.add(embedding_array)


faiss.write_index(index, "vector_index.index")

np.save("concept_names.npy", np.array(concept_names))


C:\Users\annem\AppData\Local\Temp\ipykernel_22852\1339461616.py:9: DtypeWarning: Columns (0,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path, sep='\t', quoting=3, header=None)


In [ ]:
# Prepare lists to store embeddings and concept names
embeddings = []
concept_names = []

# Process all concepts
total_concepts = len(df_selected)
for i, concept_name in enumerate(df_selected['concept_name']):
    if isinstance(concept_name, str):
        # Tokenize and obtain embeddings
        input_ids = tokenizer(concept_name, return_tensors='pt').input_ids

        with torch.no_grad():
            concept_embedding = model(input_ids)['last_hidden_state'].mean(dim=1).squeeze().numpy()

        embeddings.append(concept_embedding)
        concept_names.append(concept_name)

        # Display progress
        print(f"Processed concept {i + 1}/{total_concepts}: {concept_name}")
    else:
        print(f"Skipping concept_name: {concept_name} as it is not a valid string.")

# Convert the embeddings list to a numpy array
embedding_array = np.array(embeddings, dtype=np.float32)

# Create the Faiss index
dimension = len(embedding_array[0])
index = faiss.IndexFlatL2(dimension)
index.add(embedding_array)

# Save the Faiss index
faiss.write_index(index, "vector_index.index")

# Save the concept names for each index
np.save("concept_names.npy", np.array(concept_names))